https://www.kaggle.com/c/competitive-data-science-predict-future-sales/


In [97]:
import pandas as pd
import numpy as np

In [77]:
data = pd.read_csv("./data/sales_train.csv")

In [78]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [79]:
data = data.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].sum().reset_index(name="item_cnt_month")

In [80]:
unique_shop_id = data["shop_id"].unique()
unique_item_id = data["item_id"].unique()
unique_date_block_num = data["date_block_num"].unique()

In [81]:
from itertools import product

In [82]:
date_block_shop_item_grid = pd.DataFrame([(shop_id, item_id, date_block) for shop_id, item_id, date_block in product(unique_shop_id, unique_item_id, unique_date_block_num)], columns=["shop_id", "item_id", "date_block_num"])

In [83]:
date_block_shop_item_grid = date_block_shop_item_grid.merge(data, on=["shop_id", "item_id", "date_block_num"], how="left")

In [84]:
date_block_shop_item_grid.fillna({"item_cnt_month": 0}, inplace=True)

In [85]:
date_block_shop_item_grid.head()

,shop_id,item_id,date_block_num,item_cnt_month
0,0,32,0,6.0
1,0,32,1,10.0
2,0,32,2,0.0
3,0,32,3,0.0
4,0,32,4,0.0


In [86]:
folds = [(list(range(25,31)),[31]), (list(range(26,32)),[32]), (list(range(27,33)),[33])]

In [87]:
from sklearn.metrics import mean_squared_error

## previuos period

In [88]:
metrics = []

In [98]:
for train_range, validation_range in  folds:
    train_data = date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(train_range)]
    valid_data =  date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(validation_range)]
    train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
    train_data['prev_value'] = train_data['prev_value'].fillna(value=0)
    valid_data['prev_value'] = valid_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
    valid_data['prev_value'] = valid_data['prev_value'].fillna(value=0)
    clipped_pred =  np.clip(valid_data['prev_value'], 0, 20)
    clipped_valid = np.clip(valid_data['item_cnt_month'], 0, 20)
    error = mean_squared_error(clipped_valid, clipped_pred)
    metrics.append(error)

/tmp/ipykernel_427048/2660704190.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
/tmp/ipykernel_427048/2660704190.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prev_value'] = train_data['prev_value'].fillna(value=0)
/tmp/ipykernel_427048/2660704190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [99]:
metrics

[0.19874657984439248, 0.2342535271548891, 0.24714311918191406]

In [90]:
train_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'item_cnt_month'], dtype='object')

In [91]:
train_data.groupby(['shop_id', 'item_id', 'date_block_num']).head(10)

,shop_id,item_id,date_block_num,item_cnt_month
27,0,32,27,0.0
28,0,32,28,0.0
29,0,32,29,0.0
30,0,32,30,0.0
31,0,32,31,0.0
...,...,...,...,...
44486274,36,15925,28,0.0
44486275,36,15925,29,0.0
44486276,36,15925,30,0.0
44486277,36,15925,31,0.0


In [92]:
groups = train_data.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_month'].mean()

In [93]:
train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()

/tmp/ipykernel_427048/1206171176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()


In [94]:
train_data[(train_data.shop_id == 2) & (train_data.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,prev_value
1482903,2,32,27,0.0,NaN
1482904,2,32,28,0.0,0.0
1482905,2,32,29,0.0,0.0
1482906,2,32,30,0.0,0.0
1482907,2,32,31,1.0,0.0
1482908,2,32,32,0.0,1.0


In [96]:
train_data[train_data.prev_value > 1]

,shop_id,item_id,date_block_num,item_cnt_month,prev_value
1486478,2,1523,32,0.0,2.0
1487630,2,1905,28,1.0,2.0
1488448,2,2252,30,1.0,2.0
1488450,2,2252,32,1.0,5.0
1488719,2,2308,29,3.0,3.0
...,...,...,...,...,...
41480100,34,13285,32,0.0,2.0
41484554,34,1540,32,3.0,4.0
41484656,34,3678,32,1.0,3.0
41485030,34,11354,32,3.0,4.0
